In [178]:
import numpy as np
import pandas as pd
from tabulate import tabulate
from wmfdata import hive
from wmfdata.utils import df_to_remarkup, pct_str

# Events by platform and registration status

In [221]:
hive.run("""
select
    date_format(dt, "yyyy-MM-dd") as date,
    sum(cast(event.platform = "desktop" and event.user_id != 0 as int)) as registered_desktop,
    sum(cast(event.platform = "desktop" and event.user_id = 0 as int)) as anonymous_desktop,
    sum(cast(event.platform = "phone" and event.user_id != 0 as int)) as registered_phone,
    sum(cast(event.platform = "phone" and event.user_id = 0 as int)) as anonymous_phone
from event.editattemptstep
where
    year = 2019 and (
        (month = 6 and day > 23) or
        (month = 7)
    )
group by date_format(dt, "yyyy-MM-dd")
order by date
limit 100
""")

,date,registered_desktop,anonymous_desktop,registered_phone,anonymous_phone
0,2019-06-24,108487,234554,33555,299146
1,2019-06-25,111791,222482,26893,227859
2,2019-06-26,107107,232324,55,1410
3,2019-06-27,101278,212925,11,480
4,2019-06-28,100611,211608,0,7600
5,2019-06-29,96483,179354,0,62548
6,2019-06-30,104681,189258,0,91495
7,2019-07-01,114615,227053,0,97624
8,2019-07-02,109774,227518,0,106471
9,2019-07-03,108142,226114,1268,115234


# Inits per bucket for edit count < 100

In [224]:
buckets_r = hive.run("""
select
    date_format(dt, "yyyy-MM-dd") as date,
    event.bucket as bucket,
    count(*) as events
from event.editattemptstep
where
    event.platform = "phone" and
    event.action = "init" and
    event.user_editcount < 100 and
    wiki in (
        'azwiki', 'bgwiki', 'zh_yuewiki', 'cawiki', 'hrwiki',
        'dawiki', 'etwiki', 'fiwiki', 'elwiki', 'huwiki',
        'mswiki', 'mlwiki', 'nowiki', 'ptwiki', 'rowiki',
        'srwiki', 'svwiki', 'tawiki', 'thwiki', 'urwiki'
    ) and
    year = 2019 and (
        (month = 6 and day > 23) or
        (month = 7)
    )
group by date_format(dt, "yyyy-MM-dd"), event.bucket
""")

In [225]:
buckets = (
    buckets_r
    .pivot(index="date", columns="bucket")
    .fillna(0)
    .xs("events", axis=1)
    .applymap(int)
    .rename({np.NaN: "no bucket"}, axis=1)
    .reset_index()
    .rename_axis(None, axis=1)
)

buckets

,date,no bucket,default-source,default-visual
0,2019-06-24,6422,0,0
1,2019-06-25,5107,0,0
2,2019-06-26,23,0,0
3,2019-06-27,10,0,0
4,2019-06-28,2,220,2374
5,2019-06-29,1,1785,20300
6,2019-06-30,1,2598,29779
7,2019-07-01,0,2788,32164
8,2019-07-02,0,2932,35256
9,2019-07-03,0,3034,35754


Again without oversampling:

In [227]:
buckets_no_oversample_r = hive.run("""
select
    date_format(dt, "yyyy-MM-dd") as date,
    event.bucket as bucket,
    count(*) as events
from event.editattemptstep
where
    event.platform = "phone" and
    event.action = "init" and
    event.user_editcount < 100 and
    not event.is_oversample and
    wiki in (
        'azwiki', 'bgwiki', 'zh_yuewiki', 'cawiki', 'hrwiki',
        'dawiki', 'etwiki', 'fiwiki', 'elwiki', 'huwiki',
        'mswiki', 'mlwiki', 'nowiki', 'ptwiki', 'rowiki',
        'srwiki', 'svwiki', 'tawiki', 'thwiki', 'urwiki'
    ) and
    year = 2019 and (
        (month = 6 and day > 23) or
        (month = 7)
    )
group by date_format(dt, "yyyy-MM-dd"), event.bucket
""")

In [228]:
buckets_no_oversample = (
    buckets_no_oversample_r
    .pivot(index="date", columns="bucket")
    .fillna(0)
    .xs("events", axis=1)
    .applymap(int)
    .rename({np.NaN: "no bucket"}, axis=1)
    .reset_index()
    .rename_axis(None, axis=1)
)

buckets_no_oversample

,date,no bucket,default-source,default-visual
0,2019-06-24,5252,0,0
1,2019-06-25,4111,0,0
2,2019-06-26,21,0,0
3,2019-06-27,6,0,0
4,2019-06-28,2,167,152
5,2019-06-29,0,1431,1309
6,2019-06-30,1,2134,1858
7,2019-07-01,0,2303,2132
8,2019-07-02,0,2489,2200
9,2019-07-03,0,2527,2259


# Inits per bucket with ≥ 100 edits

In [ ]:
buckets_experienced_r = hive.run("""
select
    date_format(dt, "yyyy-MM-dd") as date,
    event.bucket as bucket,
    count(*) as events
from event.editattemptstep
where
    event.platform = "phone" and
    event.action = "init" and
    event.user_editcount >= 100 and
    wiki in (
        'azwiki', 'bgwiki', 'zh_yuewiki', 'cawiki', 'hrwiki',
        'dawiki', 'etwiki', 'fiwiki', 'elwiki', 'huwiki',
        'mswiki', 'mlwiki', 'nowiki', 'ptwiki', 'rowiki',
        'srwiki', 'svwiki', 'tawiki', 'thwiki', 'urwiki'
    ) and
    year = 2019 and (
        (month = 6 and day > 23) or
        (month = 7)
    )
group by date_format(dt, "yyyy-MM-dd"), event.bucket
""")

In [232]:
index_without_gaps = pd.date_range(
    buckets_experienced_r["date"].iloc[0], 
    buckets_experienced_r["date"].iloc[-1],
    name="date"
)

buckets_experienced= (
    buckets_experienced_r
    .assign(
        date=lambda df: pd.to_datetime(df["date"]),
        bucket=lambda df: df["bucket"].fillna("none")
    )
    .pivot(index="date", columns="bucket")
    .xs("events", axis=1)
    .fillna(0)
    .applymap(int)
    .rename(columns={"none": "no bucket"})
    .rename_axis(None, axis=1)
    .reindex(index_without_gaps, fill_value=0)
    .reset_index()
)

buckets_experienced

,date,default-source,no bucket
0,2019-06-24,0,129
1,2019-06-25,0,121
2,2019-06-26,0,1
3,2019-06-27,0,0
4,2019-06-28,0,0
5,2019-06-29,0,0
6,2019-06-30,0,0
7,2019-07-01,0,0
8,2019-07-02,0,0
9,2019-07-03,0,5


# Saves per bucket

In [238]:
saves_r = hive.run("""
select
    date_format(dt, "yyyy-MM-dd") as date,
    event.bucket as bucket,
    count(*) as events
from event.editattemptstep
where
    event.platform = "phone" and
    event.action = "saveSuccess" and
    event.user_editcount < 100 and
    wiki in (
        'azwiki', 'bgwiki', 'zh_yuewiki', 'cawiki', 'hrwiki',
        'dawiki', 'etwiki', 'fiwiki', 'elwiki', 'huwiki',
        'mswiki', 'mlwiki', 'nowiki', 'ptwiki', 'rowiki',
        'srwiki', 'svwiki', 'tawiki', 'thwiki', 'urwiki'
    ) and
    year = 2019 and (
        (month = 6 and day > 23) or
        (month = 7)
    )
group by date_format(dt, "yyyy-MM-dd"), event.bucket
""")

In [239]:
saves = (
    saves_r
    .pivot(index="date", columns="bucket")
    .fillna(0)
    .xs("events", axis=1)
    .applymap(int)
    .rename({np.NaN: "no bucket"}, axis=1)
    .reset_index()
    .rename_axis(None, axis=1)
)

saves

,date,no bucket,default-source,default-visual
0,2019-06-24,411,0,0
1,2019-06-25,347,0,0
2,2019-06-28,0,11,73
3,2019-06-29,0,148,391
4,2019-06-30,0,174,469
5,2019-07-01,0,165,437
6,2019-07-02,0,151,491
7,2019-07-03,0,188,507
8,2019-07-04,0,181,414


In [240]:
saves_no_oversample_r = hive.run("""
select
    date_format(dt, "yyyy-MM-dd") as date,
    event.bucket as bucket,
    count(*) as events
from event.editattemptstep
where
    event.platform = "phone" and
    event.action = "saveSuccess" and
    event.user_editcount < 100 and
    not event.is_oversample and
    wiki in (
        'azwiki', 'bgwiki', 'zh_yuewiki', 'cawiki', 'hrwiki',
        'dawiki', 'etwiki', 'fiwiki', 'elwiki', 'huwiki',
        'mswiki', 'mlwiki', 'nowiki', 'ptwiki', 'rowiki',
        'srwiki', 'svwiki', 'tawiki', 'thwiki', 'urwiki'
    ) and
    year = 2019 and (
        (month = 6 and day > 23) or
        (month = 7)
    )
group by date_format(dt, "yyyy-MM-dd"), event.bucket
""")

In [244]:
saves_no_oversample = (
    saves_no_oversample_r
    .pivot(index="date", columns="bucket")
    .fillna(0)
    .xs("events", axis=1)
    .applymap(int)
    .rename({np.NaN: "no bucket"}, axis=1)
    .reset_index()
    .rename_axis(None, axis=1)
)

saves_no_oversample

,date,no bucket,default-source,default-visual
0,2019-06-24,143,0,0
1,2019-06-25,107,0,0
2,2019-06-28,0,4,3
3,2019-06-29,0,56,41
4,2019-06-30,0,58,39
5,2019-07-01,0,50,43
6,2019-07-02,0,72,44
7,2019-07-03,0,69,55
8,2019-07-04,0,49,41


# Correspondence between buckets and editors

In [233]:
default_editor_inits_r = hive.run("""
select
    date_format(dt, "yyyy-MM-dd") as date,
    sum(cast(
        array(event.bucket, event.editor_interface) in (
            array("default-source", "wikitext"),
            array("default-visual", "visualeditor")
        )
    as int)) / count(*) as default_editor_inits
from event.editattemptstep
where
    event.platform = "phone" and
    event.action = "init" and
    event.bucket in ("default-source", "default-visual") and
    wiki in (
        'azwiki', 'bgwiki', 'zh_yuewiki', 'cawiki', 'hrwiki',
        'dawiki', 'etwiki', 'fiwiki', 'elwiki', 'huwiki',
        'mswiki', 'mlwiki', 'nowiki', 'ptwiki', 'rowiki',
        'srwiki', 'svwiki', 'tawiki', 'thwiki', 'urwiki'
    ) and
    year = 2019 and (
        (month = 6 and day >= 28) or
        (month = 7)
    )
group by date_format(dt, "yyyy-MM-dd")
""")

In [236]:
default_editor_inits = (
    default_editor_inits_r
    .assign(
        default_editor_inits=lambda df: df["default_editor_inits"].map(pct_str)
    ).sort_values("date")
)

default_editor_inits

,date,default_editor_inits
0,2019-06-28,97.6%
3,2019-06-29,98.1%
4,2019-06-30,98.4%
1,2019-07-01,98.4%
5,2019-07-02,98.6%
2,2019-07-03,98.5%
6,2019-07-04,97.9%


# Buckets per user

In [252]:
multiple_buckets_r = hive.run("""
select
    user_type,
    sum(cast(distinct_buckets > 1 as int)) / count(*) as multiple_buckets
from (
    select
        if(event.user_id = 0, "anonymous", "registered") as user_type,
        count(distinct event.bucket) as distinct_buckets
    from event.editattemptstep
    where
        event.platform = "phone" and
        wiki in (
            'azwiki', 'bgwiki', 'zh_yuewiki', 'cawiki', 'hrwiki',
            'dawiki', 'etwiki', 'fiwiki', 'elwiki', 'huwiki',
            'mswiki', 'mlwiki', 'nowiki', 'ptwiki', 'rowiki',
            'srwiki', 'svwiki', 'tawiki', 'thwiki', 'urwiki'
        ) and
        year = 2019 and (
            (month = 6 and day > 23) or
            (month = 7)
        )
    group by
        wiki,
        if(event.user_id = 0, event.anonymous_user_token, event.user_id),
        if(event.user_id = 0, "anonymous", "registered")
) buckets_per_user
group by user_type
""")

In [253]:
multiple_buckets = multiple_buckets_r.assign(
    multiple_buckets=lambda df: df["multiple_buckets"].map(pct_str)
)

multiple_buckets

,user_type,multiple_buckets
0,anonymous,0.0%
1,registered,0.0%


# Anonymous editors with tokens

In [214]:
anons_with_tokens_r = hive.run("""
select
    date_format(dt, "yyyy-MM-dd") as date,
    sum(cast(event.anonymous_user_token is not null as int)) / count(*) as anonymous_users_with_tokens
from event.editattemptstep
where
    event.platform = "phone" and
    event.user_id = 0 and
    wiki in (
        'azwiki', 'bgwiki', 'zh_yuewiki', 'cawiki', 'hrwiki',
        'dawiki', 'etwiki', 'fiwiki', 'elwiki', 'huwiki',
        'mswiki', 'mlwiki', 'nowiki', 'ptwiki', 'rowiki',
        'srwiki', 'svwiki', 'tawiki', 'thwiki', 'urwiki'
    ) and
    year = 2019 and (
        (month = 6 and day > 23) or
        (month = 7)
    )
group by date_format(dt, "yyyy-MM-dd")
""")

In [216]:
anons_with_tokens_r.sort_values("date").assign(
    anonymous_users_with_tokens=lambda df: df["anonymous_users_with_tokens"].map(pct_str)
)

,date,anonymous_users_with_tokens
0,2019-06-24,0.0%
4,2019-06-25,0.0%
7,2019-06-26,0.0%
1,2019-06-27,0.0%
5,2019-06-28,99.9%
8,2019-06-29,100.0%
2,2019-06-30,100.0%
9,2019-07-01,100.0%
3,2019-07-02,100.0%
6,2019-07-03,100.0%


# New revision IDs in saveSuccess events

In [258]:
rev_ids_r = hive.run("""
with saves as (
    select
        dt,
        event.editing_session_id as attempt_id,
        event.revision_id as revision_id,
        event.platform as platform,
        event.editor_interface as editor
    from event.editattemptstep
    where
        event.action = "saveSuccess" and
        year = 2019 and (
            (month = 6 and day > 23) or
            (month = 7)
        ) and
        -- Remove Flow and other non-standard edits
        event.integration = "page"
),
pre_saves as (
    select
        event.editing_session_id as attempt_id,
        max(event.revision_id) as max_revision_id
    from event.editattemptstep
    where
        event.action != "saveSuccess" and
        year = 2019 and (
            (month = 6 and day > 23) or
            (month = 7)
        ) and
        -- Remove Flow and other non-standard edits
        event.integration = "page"
    group by event.editing_session_id
)
select
    date_format(dt, "yyyy-MM-dd") as date,
    platform,
    editor,
    sum(cast(saves.revision_id > pre_saves.max_revision_id as int)) / count(*) as save_has_greater_revision_id
from saves 
left join pre_saves
on saves.attempt_id = pre_saves.attempt_id
group by
    date_format(dt, "yyyy-MM-dd"),
    platform,
    editor
""")

In [269]:
rev_ids = (
    rev_ids_r
    .assign(
        editor=lambda df: df["platform"] + " " + df["editor"]
    )
    .drop("platform", axis=1)
    .pivot(index="date", columns="editor")
    .fillna(0)
    .xs("save_has_greater_revision_id", axis=1)
    .applymap(pct_str)
    .reset_index()
)

rev_ids

editor,date,desktop visualeditor,desktop wikitext,desktop wikitext-2017,phone visualeditor,phone wikitext
0,2019-06-24,0.0%,96.4%,0.0%,0.0%,0.0%
1,2019-06-25,0.0%,95.6%,0.0%,0.0%,0.0%
2,2019-06-26,0.0%,95.9%,0.0%,0.0%,0.0%
3,2019-06-27,0.0%,95.9%,0.0%,0.0%,0.0%
4,2019-06-28,0.0%,96.1%,0.0%,0.0%,0.0%
5,2019-06-29,0.0%,96.7%,0.0%,0.0%,0.0%
6,2019-06-30,0.0%,96.8%,0.0%,0.0%,0.0%
7,2019-07-01,0.0%,96.2%,0.0%,0.0%,0.0%
8,2019-07-02,0.0%,96.1%,0.0%,0.0%,0.0%
9,2019-07-03,0.0%,96.4%,0.0%,0.0%,0.0%


# Editor switching

In [285]:
switches_r = hive.run("""
select
    date,
    editor,
    sum(cast(switches >= 1 as int)) / count(*) as sessions_with_switches
from (
    select
        date_format(dt, "yyyy-MM-dd") as date,
        event.editing_session_id as editingsessionid,
        concat(event.platform, " ", event.editor_interface) as editor
    from event.editattemptstep
    where
        event.action = "ready" and
        year = 2019 and (
            (month = 6 and day > 23) or
            (month = 7)
        )
) readies
left join (
    select
        event.editingsessionid as editingsessionid,
        count(*) as switches
    from event.visualeditorfeatureuse
    where
        event.feature = "editor-switch" and
        year = 2019 and (
            (month = 6 and day > 23) or
            (month = 7)
        )
    group by event.editingsessionid
) switches
on readies.editingsessionid = switches.editingsessionid
group by date, editor
""")

In [292]:
switches = (
    switches_r
    .pivot(index="date", columns="editor")
    .fillna(0)
    .applymap(pct_str)
    .xs("sessions_with_switches", axis=1)
    .reset_index("date")
)

switches

editor,date,desktop visualeditor,desktop wikitext,desktop wikitext-2017,phone visualeditor,phone wikitext
0,2019-06-24,0.0%,0.0%,0.0%,0.0%,0.0%
1,2019-06-25,0.0%,0.0%,0.0%,0.0%,0.0%
2,2019-06-26,0.0%,0.0%,0.0%,0.0%,0.0%
3,2019-06-27,0.0%,0.0%,0.0%,0.0%,0.0%
4,2019-06-28,0.0%,0.0%,0.0%,1.5%,0.0%
5,2019-06-29,0.0%,0.0%,0.0%,1.1%,0.9%
6,2019-06-30,0.0%,0.0%,0.0%,0.9%,0.6%
7,2019-07-01,0.0%,0.0%,0.0%,1.1%,0.8%
8,2019-07-02,0.0%,0.0%,0.0%,1.2%,0.8%
9,2019-07-03,0.0%,0.0%,0.0%,1.1%,0.7%
